# Set up example code

## Model setup

In [ ]:
import subprocess

from sklearn.gaussian_process import GaussianProcessRegressor as GPR
from sklearn.gaussian_process import kernels
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import matplotlib.cm as cm
import matplotlib.pyplot as plt

from scipy.linalg import lapack
from scipy import stats
import emcee
import numpy as np

import importlib

import os
import pickle
from pathlib import Path

import src.reader as Reader

import pandas as pd

## Step 1: prepare input pickle file

### Load stuff from text files

Input Directory

In [ ]:
indir = '/scratch/user/cameron.parker/newJETSCAPE/JETSCAPE/runs/LEP-10-2/QVir_Analysis/'

Prepare to read design to read in

In [ ]:
df = pd.read_csv(indir+'parameters.txt')
designlines = ["# 2760\n","# Version 1.0\n","# Parameter ",df.to_string(index=False)]

designfile = open("input/design.txt",'w')
designfile.writelines(designlines)
designfile.close()

RawDesign = Reader.ReadDesign("input/design.txt")
print(RawDesign)

In [ ]:
# Read data files
RawData1 = Reader.ReadData('/scratch/user/cameron.parker/newJETSCAPE/STAT/input/data/epem/charged-xp.dat')
RawData2 = Reader.ReadData('/scratch/user/cameron.parker/newJETSCAPE/STAT/input/data/epem/pion-xp.dat')
RawData3 = Reader.ReadData('/scratch/user/cameron.parker/newJETSCAPE/STAT/input/data/epem/kaon-xp.dat')
RawData4 = Reader.ReadData('/scratch/user/cameron.parker/newJETSCAPE/STAT/input/data/epem/proton-xp.dat')
RawData5 = Reader.ReadData('/scratch/user/cameron.parker/newJETSCAPE/STAT/input/data/epem/mult.dat')
RawData6 = Reader.ReadData('/scratch/user/cameron.parker/newJETSCAPE/STAT/input/data/epem/thrust.dat')

# Read model prediction
RawPrediction1 = Reader.ReadPrediction(indir+'charged-xp.dat')
RawPrediction2 = Reader.ReadPrediction(indir+'pion-xp.dat')
RawPrediction3 = Reader.ReadPrediction(indir+'kaon-xp.dat')
RawPrediction4 = Reader.ReadPrediction(indir+'proton-xp.dat')
RawPrediction5 = Reader.ReadPrediction(indir+'mult.dat')
RawPrediction6 = Reader.ReadPrediction(indir+'thrust.dat')

In [ ]:
# xlabel ylabel xlog ylog
plotvars = [
    ["xp","dN/dxp","log","log"],
    ["xp","dN/dxp","log","log"],
    ["xp","dN/dxp","log","log"],
    ["xp","dN/dxp","log","log"],
    ["N","P","linear","log"],
    #["1-thrust","P","linear","linear"]
]

### Data formatting

In [ ]:
# Initialize empty dictionary
AllData = {}

# Basic information
AllData["systems"] = ["VaVa9999"]
AllData["keys"] = RawDesign["Parameter"]
AllData["labels"] = RawDesign["Parameter"]
AllData["ranges"] = [(0.95, 3.0), (0.1, 1.0), (0.1, 0.4), (0.0, 1.5), (0.2, 0.5), (0.07, 0.2), (0.0, 0.25), (0.5, 2.5)]#, (0.5, 2.0), (0.5, 2.0), (0.5, 2.0)]
AllData["observables"] = [('Vac', ['charged-xp', 
                                   'pion-xp', 
                                   'kaon-xp', 
                                   'proton-xp', 
                                   'mult', 
                                   #'thrust',
                                   ])]

# Data points
Data = {"VaVa9999": {"Vac": {"charged-xp": RawData1["Data"], 
                             "pion-xp": RawData2["Data"], 
                             "kaon-xp": RawData3["Data"], 
                             "proton-xp": RawData4["Data"], 
                             "mult": RawData5["Data"], 
                             #"thrust": RawData6["Data"],
                             }}}

# Model predictions
Prediction = {"VaVa9999": {"Vac": {"charged-xp": {"Y": RawPrediction1["Prediction"], "x": RawData1["Data"]['x']},
                                   "pion-xp": {"Y": RawPrediction2["Prediction"], "x": RawData2["Data"]['x']},
                                   "kaon-xp": {"Y": RawPrediction3["Prediction"], "x": RawData3["Data"]['x']},
                                   "proton-xp": {"Y": RawPrediction4["Prediction"], "x": RawData4["Data"]['x']},
                                   "mult": {"Y": RawPrediction5["Prediction"], "x": RawData5["Data"]['x']},
                                   #"thrust": {"Y": RawPrediction6["Prediction"], "x": RawData6["Data"]['x']},
                                   }}}


Covariance = Reader.InitializeCovariance(Data)
Covariance["VaVa9999"][("Vac", "charged-xp")][("Vac", "charged-xp")] = Reader.EstimateCovariance(RawData1, RawData1, SysLength = {"default": 0.05})
Covariance["VaVa9999"][("Vac", "pion-xp")][("Vac", "pion-xp")] = Reader.EstimateCovariance(RawData2, RawData2, SysLength = {"default": 0.05})
Covariance["VaVa9999"][("Vac", "kaon-xp")][("Vac", "kaon-xp")] = Reader.EstimateCovariance(RawData3, RawData3, SysLength = {"default": 0.05})
Covariance["VaVa9999"][("Vac", "proton-xp")][("Vac", "proton-xp")] = Reader.EstimateCovariance(RawData4, RawData4, SysLength = {"default": 0.05})
Covariance["VaVa9999"][("Vac", "mult")][("Vac", "mult")] = Reader.EstimateCovariance(RawData5, RawData5, SysLength = {"default": 0.05})
#Covariance["VaVa9999"][("Vac", "thrust")][("Vac", "thrust")] = Reader.EstimateCovariance(RawData6, RawData6, SysLength = {"default": 0.05})

# Assign data to the dictionary
AllData["design"] = RawDesign["Design"]
AllData["model"] = Prediction
AllData["data"] = Data
AllData["cov"] = Covariance
Nobs = len(AllData["observables"][0][1])

# Save to the desired pickle file
picklefile = 'input/vac-analysis.p'
with open(picklefile, 'wb') as handle:
    pickle.dump(AllData, handle, protocol = pickle.HIGHEST_PROTOCOL)

Reader.Initialize(picklefile)

### Optional: clean past files

In [ ]:
# Clean past MCMC samples
if os.path.exists('cache/mcmc_chain.hdf'):
    os.remove("cache/mcmc_chain.hdf")

# Clean past emulator
for system in AllData["systems"]:
    if os.path.exists('cache/emulator/' + system + ".pkl"):
        os.remove('cache/emulator/' + system + ".pkl")

## Step 2: run emulator

In [ ]:
! python3 -m src.emulator --retrain --npc 10

In [ ]:
from src import lazydict, emulator
EmulatorVaVa9999 = emulator.Emulator.from_cache('VaVa9999')

## Step 3: MCMC sampling

In [ ]:
if os.path.exists('cache/mcmc_chain.hdf'):
    os.remove("cache/mcmc_chain.hdf")
! python3 -m src.mcmc --nwalkers 500 --nburnsteps 500 1500 --picklefile $picklefile

## Step 4: Analyze posterior samples

In [ ]:
import src
src.Initialize()
from src import mcmc
chain = mcmc.Chain(picklefile=picklefile)
MCMCSamples = chain.load()

In [ ]:
! python3 -m src.plots posterior gp diag_emu --picklefile $picklefile

## Step 5: adding all sorts of plots

In [ ]:
with chain.dataset() as d:
    W = d.shape[0]     # number of walkers
    S = d.shape[1]     # number of steps
    N = d.shape[2]     # number of paramters
    T = int(S / 200)   # "thinning"
    A = 20 / W
    figure, axes = plt.subplots(figsize = (15, 2 * N), ncols = 1, nrows = N)
    for i, ax in enumerate(axes):
        ax.set_ylabel(AllData["labels"][i])
        for j in range(0, W):
            ax.plot(range(0, S, T), d[j, ::T, i], alpha = A)
    plt.tight_layout()
    plt.savefig('plots/MCMCSamples.pdf', dpi = 192)

Parameter Distributions

In [ ]:
from numpy import average


NDimension = len(AllData["labels"])
Ranges = np.array(AllData["ranges"]).T
scale = 2
figure, axes = plt.subplots(figsize = (scale * NDimension, scale * NDimension), ncols = NDimension, nrows = NDimension)
Names = AllData["labels"]
for i, row in enumerate(axes):
    for j, ax in enumerate(row):
        if i==j:
            y, x, _ = ax.hist(MCMCSamples[:,i], bins=50,
                    range=Ranges[:,i], histtype='step', color='green')
            ax.set_xlabel(Names[i])
            ax.set_xlim(*Ranges[:,j])

            maxindex = 0
            maxy = y.max()
            for k,thisy in enumerate(y): 
                if(thisy == maxy): 
                    maxindex = k

            xmax = (x[maxindex]+x[maxindex+1])/2
            print(AllData["labels"][i] + ": " + str(xmax))
        if i>j:
            ax.hist2d(MCMCSamples[:, j], MCMCSamples[:, i], 
                      bins=50, range=[Ranges[:,j], Ranges[:,i]], 
                      cmap='Greens')
            ax.set_xlabel(Names[j])
            ax.set_ylabel(Names[i])
            ax.set_xlim(*Ranges[:,j])
            ax.set_ylim(*Ranges[:,i])
        if i<j:
            ax.axis('off')

    
plt.tight_layout()
plt.savefig('plots/Correlation.pdf', dpi = 192)
# figure

In [ ]:
from src.myplot import makeplot

Observables

In [ ]:
Examples = MCMCSamples[np.random.choice(range(len(MCMCSamples)), 2500), :]
TempPrediction = {"VaVa9999": EmulatorVaVa9999.predict(Examples)}

makeplot(AllData, plotvars, TempPrediction, "Posteriors")

In [ ]:
Examples = AllData["design"]
TempPrediction = {"VaVa9999": EmulatorVaVa9999.predict(Examples)}

makeplot(AllData, plotvars, TempPrediction, "Predicted Design")

Priors

In [ ]:
TempPrediction = AllData["model"]

makeplot(AllData, plotvars, TempPrediction, "Priors")

In [ ]:
# close all plots to save memory
plt.close('all')